## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-07-13-00-00 +0000,missionlocal,Fence,https://missionlocal.org/2025/10/sf-photos-fence/
1,2025-10-07-12-59-33 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/07/us/tru...
2,2025-10-07-12-58-18 +0000,bbc,Canadian marine park threatens to euthanise 30...,https://www.bbc.com/news/articles/c8jmddp1v49o...
3,2025-10-07-12-57-47 +0000,nyt,Supreme Court Lets Trump Revoke Deportation Pr...,https://www.nytimes.com/2025/10/03/us/politics...
4,2025-10-07-12-54-48 +0000,bbc,DPD deliveries could face disruption as driver...,https://www.bbc.com/news/articles/c701ddg4ep9o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,59
201,new,24
803,shutdown,14
417,deal,14
273,gaza,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
146,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...,129
216,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...,121
128,2025-10-07-01-27-00 +0000,wsj,President Trump signaled a willingness to stri...,https://www.wsj.com/politics/policy/trump-open...,120
258,2025-10-06-17-52-24 +0000,nypost,Retired IDF general who saved his family on Oc...,https://nypost.com/2025/10/06/world-news/retir...,114
180,2025-10-06-21-45-09 +0000,latimes,Trump advisors amp up extreme rhetoric against...,https://www.latimes.com/politics/story/2025-10...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
146,129,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...
153,75,2025-10-06-22-37-00 +0000,wsj,The Reserve Bank of New Zealand has establishe...,https://www.wsj.com/articles/new-zealand-centr...
216,62,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...
226,60,2025-10-06-19-34-39 +0000,cbc,Bari Weiss named editor-in-chief of CBS News u...,https://www.cbc.ca/news/entertainment/bari-wei...
127,52,2025-10-07-01-58-41 +0000,nyt,Israel and Hamas Meet in Egypt to Discuss Gaza...,https://www.nytimes.com/2025/10/06/world/middl...
231,40,2025-10-06-19-11-11 +0000,bbc,US Supreme Court rejects Ghislaine Maxwell app...,https://www.bbc.com/news/articles/cn83e6q7lq4o...
276,38,2025-10-06-16-48-36 +0000,bbc,Sudan militia leader convicted of war crimes d...,https://www.bbc.com/news/articles/cly1egd5d0vo...
243,38,2025-10-06-18-35-07 +0000,cbc,Ex-NFL QB Mark Sanchez faces felony battery ch...,https://www.cbc.ca/sports/football/nfl/mark-sa...
99,37,2025-10-07-06-11-52 +0000,nypost,Family of engineer found dead in Connecticut c...,https://nypost.com/2025/10/07/us-news/family-o...
126,34,2025-10-07-02-00-55 +0000,nypost,California’s Hollywood Burbank Airport to go h...,https://nypost.com/2025/10/06/us-news/californ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
